In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/output_pln_usd.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
scaler = scaler.fit(df_numpy[:3000])
df_scalled = scaler.transform(df_numpy)
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[0:2500]
df_val = df_scalled[2500:3000] #300
df_game = df_scalled[3001:]

In [ ]:
lookback = 30 #15
step = 1
delay = 30 #0
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')
float_train_data = np.array(df_train).astype('float32')
float_val_data = np.array(df_val).astype('float32')
float_game_data = np.array(df_game).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_train_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2499 -delay,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_val_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=500-delay ,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_game_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=540 -delay,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2500 - lookback)
val_steps = (500 - lookback)
test_steps = (541 - lookback)

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  input_shape=(None, float_train_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 24s 226ms/step - loss: 0.2632 - val_loss: 0.4658
Epoch 2/200
100/100 [==============================] - 22s 221ms/step - loss: 0.1244 - val_loss: 0.1844
Epoch 3/200
100/100 [==============================] - 21s 215ms/step - loss: 0.0977 - val_loss: 0.1032
Epoch 4/200
100/100 [==============================] - 22s 217ms/step - loss: 0.0947 - val_loss: 0.0661
Epoch 5/200
100/100 [==============================] - 21s 216ms/step - loss: 0.0811 - val_loss: 0.1541
Epoch 6/200
100/100 [==============================] - 21s 216ms/step - loss: 0.0811 - val_loss: 0.0588
Epoch 7/200
100/100 [==============================] - 23s 236ms/step - loss: 0.0792 - val_loss: 0.0449
Epoch 8/200
100/100 [==============================] - 22s 218ms/step - loss: 0.0795 - val_loss: 0.0588
Epoch 9/200
100/100 [==============================] - 21s 216ms/step - loss: 0.0757 - val_loss: 0.1568
Epoch 10/200
100/100 [==============================] - 22s 217m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 18s 36ms/step - loss: 0.0546


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_lstm_usd_pln.h5")

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.GRU(140,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(120,
  activation='sigmoid',))
model.add(layers.GRU(2,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 30s 272ms/step - loss: 0.2026 - val_loss: 0.2771
Epoch 2/200
100/100 [==============================] - 28s 280ms/step - loss: 0.1890 - val_loss: 0.1987
Epoch 3/200
100/100 [==============================] - 27s 267ms/step - loss: 0.1083 - val_loss: 0.0469
Epoch 4/200
100/100 [==============================] - 28s 286ms/step - loss: 0.0939 - val_loss: 0.0466
Epoch 5/200
100/100 [==============================] - 27s 268ms/step - loss: 0.0851 - val_loss: 0.1480
Epoch 6/200
100/100 [==============================] - 27s 269ms/step - loss: 0.0820 - val_loss: 0.0459
Epoch 7/200
100/100 [==============================] - 27s 270ms/step - loss: 0.0803 - val_loss: 0.0467
Epoch 8/200
100/100 [==============================] - 27s 268ms/step - loss: 0.0802 - val_loss: 0.0461
Epoch 9/200
100/100 [==============================] - 27s 269ms/step - loss: 0.0769 - val_loss: 0.0913
Epoch 10/200
100/100 [==============================] - 27s 274m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 22s 42ms/step - loss: 0.0634


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_gru_usd_pln.h5")

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.GRU(5,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid'))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 30s 274ms/step - loss: 0.1760 - val_loss: 0.5385
Epoch 2/200
100/100 [==============================] - 26s 263ms/step - loss: 0.1650 - val_loss: 0.3825
Epoch 3/200
100/100 [==============================] - 25s 255ms/step - loss: 0.1643 - val_loss: 0.1223
Epoch 4/200
100/100 [==============================] - 25s 256ms/step - loss: 0.1631 - val_loss: 0.2555
Epoch 5/200
100/100 [==============================] - 25s 253ms/step - loss: 0.1671 - val_loss: 0.3738
Epoch 6/200
100/100 [==============================] - 25s 255ms/step - loss: 0.1674 - val_loss: 0.2620
Epoch 7/200
100/100 [==============================] - 25s 255ms/step - loss: 0.1553 - val_loss: 0.0569
Epoch 8/200
100/100 [==============================] - 27s 276ms/step - loss: 0.1408 - val_loss: 0.2514
Epoch 9/200
100/100 [==============================] - 26s 258ms/step - loss: 0.1154 - val_loss: 0.2218
Epoch 10/200
100/100 [==============================] - 26s 257m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 20s 39ms/step - loss: 0.0526


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_hybrid_usd_pln.h5")